In [7]:
# Telecom X - Análise de Evasão de Clientes (Churn)
# Código completo corrigido para importação, limpeza, análise e relatório

import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações gerais para gráficos
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (8, 5)

# 1. Importação dos dados via requests para garantir leitura correta
url = "https://raw.githubusercontent.com/alura-cursos/challenge2-data-science/refs/heads/main/TelecomX_Data.json"

response = requests.get(url)
json_data = response.json()

# Verificar estrutura do JSON
if isinstance(json_data, dict):
    # Se tiver uma chave que contenha os dados (ajuste conforme necessário)
    possible_keys = ['data', 'records', 'items']
    found_key = None
    for key in possible_keys:
        if key in json_data:
            found_key = key
            break

    if found_key:
        df = pd.DataFrame(json_data[found_key])
    else:
        # Caso JSON seja um dict simples (ou não contenha lista dentro)
        df = pd.DataFrame([json_data])
else:
    # Se for lista de dicts
    df = pd.DataFrame(json_data)

# Mostrar colunas para confirmar
print("Colunas do DataFrame:", df.columns.tolist())

# 2. Limpeza e tratamento dos dados

# Ajustar espaços em branco nos nomes das colunas e valores
df.columns = df.columns.str.strip()
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Converter TotalCharges para numérico, tratar erros
if 'TotalCharges' in df.columns:
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
    # Remover linhas com TotalCharges nulo
    df.dropna(subset=['TotalCharges'], inplace=True)
else:
    print("Coluna 'TotalCharges' não encontrada.")

# Remover duplicatas
df.drop_duplicates(inplace=True)

# Criar coluna Contas_Diarias
if 'MonthlyCharges' in df.columns:
    df['Contas_Diarias'] = (df['MonthlyCharges'] / 30).round(2)
else:
    print("Coluna 'MonthlyCharges' não encontrada.")

# 3. Análise exploratória de dados

# Verificar se a coluna 'Churn' existe
if 'Churn' not in df.columns:
    raise KeyError("Coluna 'Churn' não encontrada no DataFrame.")

# 3.1 Distribuição da variável alvo - Churn
print("\nDistribuição de Churn (evasão):")
print(df['Churn'].value_counts())
plt.figure(figsize=(6,4))
sns.countplot(x='Churn', data=df, palette='Set2')
plt.title('Distribuição de Clientes por Churn')
plt.show()

# 3.2 Análise de churn por variáveis categóricas
categoricas = ['gender', 'Contract', 'PaymentMethod', 'InternetService', 'SeniorCitizen']
for col in categoricas:
    if col in df.columns:
        plt.figure(figsize=(8,4))
        ct = pd.crosstab(df[col], df['Churn'], normalize='index') * 100
        ct.plot(kind='bar', stacked=True, color=['#66b3ff', '#ff6666'], ax=plt.gca())
        plt.title(f'Percentual de Churn por {col}')
        plt.xlabel(col)
        plt.ylabel('Porcentagem (%)')
        plt.xticks(rotation=45)
        plt.show()
    else:
        print(f"Coluna '{col}' não encontrada para análise categórica.")

# 3.3 Análise das variáveis numéricas por churn
numericas = ['TotalCharges', 'tenure', 'MonthlyCharges', 'Contas_Diarias']
for col in numericas:
    if col in df.columns:
        plt.figure(figsize=(8,4))
        sns.boxplot(x='Churn', y=col, data=df, palette='Set3')
        plt.title(f'Distribuição de {col} por Churn')
        plt.show()
    else:
        print(f"Coluna '{col}' não encontrada para análise numérica.")

print("\nEstatísticas descritivas por Churn:")
estatisticas = df.groupby('Churn')[numericas].agg(['mean', 'median', 'std']).round(2)
print(estatisticas)

# 4. Relatório resumido - texto para interpretação

print("\n--- Relatório Resumido ---\n")

print("Introdução:")
print("O objetivo deste projeto foi analisar os dados dos clientes da Telecom X para entender os fatores que levam à evasão (churn).")
print("O churn impacta negativamente a empresa, e identificar esses fatores permite ações de retenção mais eficazes.\n")

print("Limpeza e Tratamento dos Dados:")
print("- Corrigimos o tipo da coluna 'TotalCharges' quando presente.")
print("- Removemos dados nulos e duplicados.")
print("- Tratamos inconsistências nas categorias e criamos a coluna 'Contas_Diarias' para detalhar o faturamento.\n")

print("Análise Exploratória de Dados:")
print("- Clientes com contratos mensais têm maior probabilidade de evasão.")
print("- Clientes com menor tempo de contrato tendem a cancelar mais.")
print("- Faturamento mensal e total têm influência relevante na evasão.")
print("- Serviços como suporte técnico e segurança online impactam na retenção.\n")

print("Conclusões e Recomendações:")
print("- Focar em contratos de longo prazo para reduzir churn.")
print("- Investir em experiência e suporte para novos clientes.")
print("- Avaliar política de preços e planos personalizados.")
print("- Monitorar continuamente para ações rápidas de retenção.\n")

print("Fim do relatório.")


Colunas do DataFrame: ['customerID', 'Churn', 'customer', 'phone', 'internet', 'account']
Coluna 'TotalCharges' não encontrada.


/tmp/ipython-input-7-4257101590.py:45: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


TypeError: unhashable type: 'dict'